# DYLE+GPT with Product test set

In [ ]:
!pip install --upgrade openai
!pip install rouge

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from rouge import Rouge
from pprint import pprint
import json
from sys import displayhook
from openai import OpenAI
import tiktoken
import os
from rouge import Rouge
from pprint import pprint
import numpy as np


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Utils

In [ ]:
client = OpenAI(api_key='') # Insert OpenAI API key

In [ ]:
# Following QMSum
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

# For openai
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

#num_tokens_from_string(str, "cl100k_base")

# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound }', '')
    text = text.replace('{ disfmarker }', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause }', '')
    text = text.replace('{ nonvocalsound }', '')
    text = text.replace('{ gap }', '')
    return text

## Data Loading

In [ ]:
# make lists with queries
topic_list = []
queries = []
ref_list = []
ref_and_queries = {}

for file in os.listdir('../Data/QMSum/data/Product/test/'): 
    if file != '.ipynb_checkpoints':
        f = os.path.join('./Data/prod', file)
        with open(f, "r") as f:
             data = json.load(f)
        f.close()

        for i in data['topic_list']:
            for key, val in i.items():
                if key == "topic":
                    topic_list.append(val)
        for i in data['general_query_list']:
            for key, val in i.items():
                if key == "query":
                    queries.append(val)
                if key == "answer": 
                    ref_list.append(tokenize(val))
        for i in data['specific_query_list']:
            for key, val in i.items():
                if key == "query":
                    queries.append(val)
                if key == "answer": 
                    ref_list.append(tokenize(val))
                    
                    
for i in range(0, len(queries)):
    ref_and_queries[ref_list[i]] = queries[i]
    
    
assert(len(queries) == len(ref_list))
assert(len(queries) == 151)
assert(len(ref_list) == 151)
assert(len(ref_and_queries) == len(ref_list))
assert(len(ref_and_queries) == len(queries))

Getting the retrieved content from DYLE

In [ ]:
#retrieved topk
with open('../Data/Product_dyle_output.txt', 'r') as file:
    text = file.read()

In [ ]:
context_and_queries = {}
dyle_answers = []
ref_list_out = []

word_tok_text = word_tokenize(clean_data(text))
topk = []
for i in range(len(word_tok_text)):
    this_topk = []
    this_ans = []
    this_ref = []
    if word_tok_text[i] == 'Pred':
        i = i+2
        while word_tok_text[i] != '--':
            this_ans.append(word_tok_text[i])
            i = i+1
        this_ans = ' '.join(this_ans)
        dyle_answers.append(this_ans)
        
    if word_tok_text[i] == 'topk':     
        i = i+2
        while word_tok_text[i] != '--':
            this_topk.append(word_tok_text[i])
            i = i+1
        this_topk = ' '.join(this_topk)
        topk.append(this_topk)
        
    if word_tok_text[i] == 'Ref':
        i = i+2
        while word_tok_text[i] != '--':
            this_ref.append(word_tok_text[i])
            i = i+1
        this_ref = ' '.join(this_ref)
        ref_list_out.append(this_ref)


i = 0
for ref in ref_list_out:
    for ref, query in ref_and_queries.items():
        if ref == ref:
            queries[i] == query
    i += 1
           
for i in range(len(queries)):
    
    context_and_queries[clean_data(topk[i])] = queries[i]


In [ ]:
print(len(context_and_queries))

151


## Querying the model

In [ ]:
def get_info(topk: str) -> str:
    info = f"""Use the below content from a meeting transcript to answer all questions."

retrieved content:
\"\"\"
{topk}
\"\"\"

"""
    return info

In [ ]:
def ask(query: str, topk: str,
    print_message: bool = False) -> str:
   # if print_message:
    #    print(message)
    messages = [
        {"role": "system", "content": get_info(topk)},
        {"role": "user", "content": query},
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=0.0
    )
    response_message = response.choices[0].message.content
    return response_message

In [ ]:
hyp_list = []
for topk, question in context_and_queries.items():
    response = ask(question, topk)
    hyp_list.append(tokenize(response))
    
print(len(hyp_list))

151


## Evaluation

Summary evaluation

In [ ]:
print(len(hyp_list))

data = []
for i in range(len(hyp_list)):
    data.append({'hyp': hyp_list[i], 'ref': ref_list[i]})

hyps, refs = map(list, zip(*[[d['hyp'], d['ref']] for d in data]))
rouge = Rouge()

scores = rouge.get_scores(hyps, refs)
# or
avg_scores = rouge.get_scores(hyps, refs, avg=True)

pprint(avg_scores)

151
{'rouge-1': {'f': 0.22736853110443908,
             'p': 0.27615508173700654,
             'r': 0.22481599144401837},
 'rouge-2': {'f': 0.05995408940415393,
             'p': 0.07807180971984276,
             'r': 0.05932521073651497},
 'rouge-l': {'f': 0.19313297371451388,
             'p': 0.23414034366997105,
             'r': 0.1917696265820316}}


In [ ]:
with open('dyle_gpt_prod_output.txt', 'w') as f:
    for line in data:
        f.write(f"{line}\n")
f.close()

In [ ]:
print(queries[2])
print(data[2])

Did the group think the remote control was easy to use when discussing evaluation criteria of the remote control?
{'hyp': 'there is no specific mention in the content about the group discussing the evaluation criteria of the remote control or their thoughts on its ease of use .', 'ref': 'user interface believed that voice recognition was hard to learn while industrial designer argued that the remote control had two parts and everyone could use the basic part . he also pointed out that an advanced user would like to explore additional functions .'}


Token number

In [ ]:

retrieved_tok = []
gen_tok = []
i = 0
for topk, question in context_and_queries.items():
    retrieved_tok.append(num_tokens_from_string(topk, "cl100k_base"))
    gen_tok.append(num_tokens_from_string(hyp_list[i], "cl100k_base"))
    i += 1
print('average retrieved tokens DYLE: ', np.mean(retrieved_tok))
print('average generated tokens GPT: ', np.mean(gen_tok))


average retrieved tokens DYLE:  661.8278145695364
average generated tokens GPT:  76.23841059602648


In [ ]:
#average generated tokens DYLE
print('average generated tokens DYLE: ', np.mean([num_tokens_from_string(i, "cl100k_base") for i in dyle_answers]))


average generated tokens DYLE:  105.57615894039735
